# Chapter 52: Advanced Performance Techniques

Modern web applications face increasing demands for instant interactivity while handling growing data complexity. Beyond basic lazy loading and code splitting, advanced performance engineering requires deep understanding of React's rendering behavior, memory management patterns, and browser resource optimization. This chapter explores cutting-edge techniques including the React Compiler's automatic memoization, memory leak prevention in single-page applications, and sophisticated bundle optimization strategies.

By the end of this chapter, you'll leverage the React Compiler for zero-effort memoization, implement concurrent rendering patterns with transitions, diagnose and prevent memory leaks in long-running applications, analyze and optimize bundle composition using tree-shaking and dynamic imports, utilize resource hints for predictive loading, profile JavaScript execution to identify bottlenecks, and implement virtual scrolling for massive datasets without DOM overhead.

## 52.1 React Compiler Optimization

Automatic memoization without manual `useMemo` and `useCallback`.

### Understanding the React Compiler

The React Compiler (formerly React Forget) automatically memoizes components and hooks:

```typescript
// Before: Manual memoization everywhere
'use client';

import { useMemo, useCallback, memo } from 'react';

interface ExpensiveListProps {
  items: Array<{ id: string; name: string; value: number }>;
  onSelect: (id: string) => void;
  filter: string;
}

// Manual component memoization
const ExpensiveList = memo(function ExpensiveList({ 
  items, 
  onSelect, 
  filter 
}: ExpensiveListProps) {
  // Manual computation memoization
  const filteredItems = useMemo(() => {
    console.log('Filtering...');
    return items.filter(item => 
      item.name.toLowerCase().includes(filter.toLowerCase())
    );
  }, [items, filter]);

  // Manual callback memoization
  const handleClick = useCallback((id: string) => {
    onSelect(id);
  }, [onSelect]);

  return (
    <ul>
      {filteredItems.map(item => (
        <li key={item.id} onClick={() => handleClick(item.id)}>
          {item.name}: {item.value}
        </li>
      ))}
    </ul>
  );
});

// After: React Compiler handles memoization automatically
'use client';

// No manual memoization needed - compiler adds it at build time
interface ExpensiveListProps {
  items: Array<{ id: string; name: string; value: number }>;
  onSelect: (id: string) => void;
  filter: string;
}

function ExpensiveList({ items, onSelect, filter }: ExpensiveListProps) {
  // This computation is automatically memoized
  const filteredItems = items.filter(item => 
    item.name.toLowerCase().includes(filter.toLowerCase())
  );

  // This callback is automatically memoized
  const handleClick = (id: string) => {
    onSelect(id);
  };

  return (
    <ul>
      {filteredItems.map(item => (
        <li key={item.id} onClick={() => handleClick(item.id)}>
          {item.name}: {item.value}
        </li>
      ))}
    </ul>
  );
}
```

### Configuration and Setup

Enabling the compiler in Next.js:

```typescript
// next.config.js
/** @type {import('next').NextConfig} */
const nextConfig = {
  experimental: {
    reactCompiler: true,
  },
};

module.exports = nextConfig;

// For gradual adoption, configure per-file
// components/legacy-component.tsx
'use client';
'use no memo'; // Opt-out of compiler for this file

import { useState } from 'react';

export function LegacyComponent() {
  // This component won't be optimized by React Compiler
  const [count, setCount] = useState(0);
  return <button onClick={() => setCount(c => c + 1)}>{count}</button>;
}
```

### Compiler-Aware Patterns

Writing code that maximizes automatic optimization:

```typescript
// Good: Stable object references when possible
'use client';

function Dashboard({ userId }: { userId: string }) {
  // ✅ Compiler can optimize this - stable config object
  const config = {
    apiKey: process.env.NEXT_PUBLIC_API_KEY,
    endpoint: '/api/data',
  };

  // ✅ Automatically memoized
  const data = useApiData(userId, config);

  return <DashboardView data={data} />;
}

// Bad: Unstable references that break memoization
function BadDashboard({ userId }: { userId: string }) {
  // ❌ New object every render - compiler can't optimize effectively
  const options = { 
    timestamp: Date.now(), // Unstable!
    userId 
  };

  const data = useApiData(userId, options); // Re-runs every render
  return <DashboardView data={data} />;
}

// Solution: Use stable references for configuration
function GoodDashboard({ userId }: { userId: string }) {
  // ✅ Stable - only changes when userId changes
  const options = useMemo(() => ({
    userId,
    fetchPolicy: 'cache-first' as const,
  }), [userId]);

  const data = useApiData(userId, options);
  return <DashboardView data={data} />;
}
```

## 52.2 Concurrent Rendering Patterns

Using React 18+ concurrent features for improved perceived performance.

### Transitions and Priorities

Non-blocking state updates:

```typescript
'use client';

import { useState, useTransition, useDeferredValue } from 'react';
import { useRouter } from 'next/navigation';

export function SearchResults() {
  const [isPending, startTransition] = useTransition();
  const [query, setQuery] = useState('');
  
  // Deferred value allows keeping UI responsive
  const deferredQuery = useDeferredValue(query);
  
  // High priority: Update input immediately
  const handleChange = (e: React.ChangeEvent<HTMLInputElement>) => {
    setQuery(e.target.value); // Urgent update
  };
  
  // Low priority: Update results
  const handleSearch = (searchTerm: string) => {
    startTransition(() => {
      // This update can be interrupted by more urgent updates
      performExpensiveSearch(searchTerm);
    });
  };

  return (
    <div>
      <input 
        value={query} 
        onChange={handleChange}
        className="border p-2"
      />
      
      {/* Show stale data while transitioning */}
      {isPending && (
        <div className="opacity-50">
          <Results query={deferredQuery} />
        </div>
      )}
      
      {!isPending && <Results query={deferredQuery} />}
    </div>
  );
}

// Navigation transitions
export function TabContainer() {
  const [activeTab, setActiveTab] = useState('home');
  const [isPending, startTransition] = useTransition();
  
  const switchTab = (tab: string) => {
    startTransition(() => {
      setActiveTab(tab);
    });
  };
  
  return (
    <div>
      <div className="flex gap-4 border-b">
        {['home', 'profile', 'settings'].map(tab => (
          <button
            key={tab}
            onClick={() => switchTab(tab)}
            className={`
              p-4 capitalize
              ${activeTab === tab ? 'border-b-2 border-blue-500' : ''}
              ${isPending && activeTab === tab ? 'text-gray-400' : ''}
            `}
          >
            {tab}
            {isPending && activeTab === tab && (
              <span className="ml-2 inline-block animate-spin">⟳</span>
            )}
          </button>
        ))}
      </div>
      
      <div className="p-4">
        {activeTab === 'home' && <HomeContent />}
        {activeTab === 'profile' && <ProfileContent />}
        {activeTab === 'settings' && <SettingsContent />}
      </div>
    </div>
  );
}
```

### Suspense Boundaries with Streaming

Granular loading states:

```typescript
// app/page.tsx
import { Suspense } from 'react';
import { ProductSkeleton } from '@/components/skeletons';
import { Recommendations } from '@/components/recommendations';
import { Reviews } from '@/components/reviews';

export default function ProductPage() {
  return (
    <div>
      <h1>Product Details</h1>
      
      {/* Immediate render */}
      <ProductInfo />
      
      {/* Stream in non-critical content */}
      <Suspense fallback={<ProductSkeleton count={3} />}>
        <Recommendations />
      </Suspense>
      
      <Suspense fallback={<div>Loading reviews...</div>}>
        <Reviews />
      </Suspense>
    </div>
  );
}

// components/recommendations.tsx
async function Recommendations() {
  // This fetch is initiated immediately but doesn't block other content
  const products = await fetchRecommendations();
  
  return (
    <div className="grid grid-cols-3 gap-4">
      {products.map(p => (
        <ProductCard key={p.id} product={p} />
      ))}
    </div>
  );
}

// Error boundaries with Suspense
'use client';

import { Suspense, useState } from 'react';
import { ErrorBoundary } from 'react-error-boundary';

export function ResilientComponent() {
  return (
    <ErrorBoundary FallbackComponent={ErrorFallback}>
      <Suspense fallback={<Loading />}>
        <AsyncContent />
      </Suspense>
    </ErrorBoundary>
  );
}

function ErrorFallback({ error, resetErrorBoundary }: { 
  error: Error; 
  resetErrorBoundary: () => void 
}) {
  return (
    <div className="p-4 bg-red-50 text-red-600">
      <p>Something went wrong:</p>
      <pre>{error.message}</pre>
      <button onClick={resetErrorBoundary}>Try again</button>
    </div>
  );
}
```

## 52.3 Memory Leak Prevention

Managing subscriptions and references in long-running SPAs.

### Cleanup Patterns

Proper resource disposal:

```typescript
'use client';

import { useEffect, useRef, useState } from 'react';

// ❌ Bad: Missing cleanup
function BadComponent() {
  const [data, setData] = useState(null);
  
  useEffect(() => {
    const ws = new WebSocket('wss://api.example.com');
    ws.onmessage = (e) => setData(JSON.parse(e.data));
    // Missing: ws.close() cleanup
  }, []);
  
  return <div>{data}</div>;
}

// ✅ Good: Proper cleanup
function GoodComponent() {
  const [data, setData] = useState(null);
  const wsRef = useRef<WebSocket | null>(null);
  
  useEffect(() => {
    const ws = new WebSocket('wss://api.example.com');
    wsRef.current = ws;
    
    ws.onmessage = (e) => {
      setData(JSON.parse(e.data));
    };
    
    // Cleanup function
    return () => {
      ws.close();
      wsRef.current = null;
    };
  }, []);
  
  return <div>{data}</div>;
}

// AbortController for fetch cleanup
function useApiData(url: string) {
  const [data, setData] = useState(null);
  
  useEffect(() => {
    const controller = new AbortController();
    
    fetch(url, { signal: controller.signal })
      .then(r => r.json())
      .then(setData)
      .catch(err => {
        if (err.name === 'AbortError') return;
        console.error(err);
      });
    
    return () => controller.abort();
  }, [url]);
  
  return data;
}

// Event listener cleanup
function useKeyPress(targetKey: string, callback: () => void) {
  useEffect(() => {
    const handleKeyPress = (e: KeyboardEvent) => {
      if (e.key === targetKey) callback();
    };
    
    window.addEventListener('keydown', handleKeyPress);
    
    return () => {
      window.removeEventListener('keydown', handleKeyPress);
    };
  }, [targetKey, callback]);
}
```

### Refs and Closure Cleanup

Preventing stale closures and memory leaks:

```typescript
'use client';

import { useCallback, useRef, useEffect } from 'react';

// ✅ Using refs to avoid stale closures in intervals
function useInterval(callback: () => void, delay: number | null) {
  const savedCallback = useRef(callback);
  
  // Update ref when callback changes
  useEffect(() => {
    savedCallback.current = callback;
  }, [callback]);
  
  useEffect(() => {
    if (delay === null) return;
    
    const id = setInterval(() => savedCallback.current(), delay);
    return () => clearInterval(id);
  }, [delay]);
}

// ✅ Cleaning up DOM references
function useMeasure() {
  const ref = useRef<HTMLDivElement>(null);
  const [bounds, setBounds] = useState<DOMRectReadOnly>();
  
  useEffect(() => {
    if (!ref.current) return;
    
    const observer = new ResizeObserver(([entry]) => {
      setBounds(entry.contentRect);
    });
    
    observer.observe(ref.current);
    
    return () => {
      observer.disconnect();
    };
  }, []);
  
  return { ref, bounds };
}

// ✅ WeakMap for caching without preventing GC
const computationCache = new WeakMap<object, any>();

function expensiveComputation(obj: object) {
  if (computationCache.has(obj)) {
    return computationCache.get(obj);
  }
  
  const result = /* expensive work */;
  computationCache.set(obj, result);
  return result;
}
```

## 52.4 Bundle Optimization

Analyzing and reducing JavaScript payload.

### Bundle Analysis

Identifying bloat:

```typescript
// next.config.js
const withBundleAnalyzer = require('@next/bundle-analyzer')({
  enabled: process.env.ANALYZE === 'true',
});

module.exports = withBundleAnalyzer({
  // webpack optimization
  webpack: (config, { isServer }) => {
    if (!isServer) {
      config.optimization = {
        ...config.optimization,
        usedExports: true, // Tree shaking
        sideEffects: false,
        splitChunks: {
          chunks: 'all',
          cacheGroups: {
            vendor: {
              test: /[\\/]node_modules[\\/](react|react-dom)[\\/]/,
              name: 'vendor',
              chunks: 'all',
              priority: 40,
            },
            commons: {
              name: 'commons',
              chunks: 'initial',
              minChunks: 2,
              priority: 20,
            },
          },
        },
      };
    }
    return config;
  },
});

// Analyzing specific imports
// scripts/analyze-imports.js
const fs = require('fs');
const path = require('path');

function analyzeImports(dir) {
  const files = fs.readdirSync(dir, { recursive: true });
  const imports = {};
  
  files.filter(f => f.endsWith('.tsx')).forEach(file => {
    const content = fs.readFileSync(path.join(dir, file), 'utf-8');
    const matches = content.match(/import\s+.*\s+from\s+['"]([^'"]+)['"]/g) || [];
    
    matches.forEach(imp => {
      const pkg = imp.match(/from\s+['"]([^'"]+)['"]/)?.[1];
      if (pkg && !pkg.startsWith('.') && !pkg.startsWith('@/')) {
        imports[pkg] = (imports[pkg] || 0) + 1;
      }
    });
  });
  
  console.log('External dependencies usage:');
  Object.entries(imports)
    .sort((a, b) => b[1] - a[1])
    .forEach(([pkg, count]) => console.log(`  ${pkg}: ${count} files`));
}

analyzeImports('./src');
```

### Tree-Shaking Optimization

Ensuring dead code elimination:

```typescript
// ✅ Good: ES modules with sideEffects: false
// package.json
{
  "sideEffects": false,
  "module": "esm/index.js",
  "main": "cjs/index.js"
}

// ✅ Good: Named exports for tree-shaking
// lib/utils.ts
export function formatDate(date: Date): string {
  return date.toLocaleDateString();
}

export function formatCurrency(amount: number): string {
  return `$${amount.toFixed(2)}`;
}

// Only formatDate is included if that's all you import
import { formatDate } from '@/lib/utils';

// ❌ Bad: Default exports with methods
// lib/helpers.ts
export default {
  formatDate: (d: Date) => d.toLocaleDateString(),
  formatCurrency: (n: number) => `$${n.toFixed(2)}`,
  heavyFunction: () => { /* 100kb of code */ }
};

// Imports entire object even if only using one method
import helpers from '@/lib/helpers';

// ✅ Good: Barrel files with selective exports
// components/index.ts
export { Button } from './Button';
export { Input } from './Input';
// Heavy component only exported explicitly
export { HeavyChart } from './HeavyChart';

// Import only what you need
import { Button } from '@/components';
```

### Dynamic Imports with Preloading

Strategic code splitting:

```typescript
'use client';

import { useState, useEffect } from 'react';
import dynamic from 'next/dynamic';

// Preload when browser is idle
const HeavyChart = dynamic(() => import('./HeavyChart'), {
  loading: () => <div>Loading chart...</div>,
  ssr: false, // Only client-side
});

// Preload on interaction
function Dashboard() {
  const [showChart, setShowChart] = useState(false);
  const [ChartComponent, setChartComponent] = useState<any>(null);
  
  // Preload on hover
  const preloadChart = () => {
    if (!ChartComponent) {
      import('./HeavyChart').then(mod => {
        setChartComponent(() => mod.default);
      });
    }
  };
  
  return (
    <div>
      <button 
        onMouseEnter={preloadChart}
        onClick={() => setShowChart(true)}
      >
        Show Analytics
      </button>
      
      {showChart && (ChartComponent ? <ChartComponent /> : <div>Loading...</div>)}
    </div>
  );
}

// Route-based dynamic imports
// app/dashboard/page.tsx
import { Suspense } from 'react';

const HeavyWidget = dynamic(() => import('@/components/HeavyWidget'), {
  suspense: true,
});

export default function Dashboard() {
  return (
    <div>
      <h1>Dashboard</h1>
      <Suspense fallback={<div>Loading widget...</div>}>
        <HeavyWidget />
      </Suspense>
    </div>
  );
}
```

## 52.5 Network Optimization

Resource hints and predictive loading.

### Resource Hints

Preloading critical assets:

```typescript
// app/layout.tsx
import type { Metadata } from 'next';

export const metadata: Metadata = {
  // Preconnect to origins
  other: {
    'link': [
      { rel: 'preconnect', href: 'https://fonts.googleapis.com' },
      { rel: 'preconnect', href: 'https://fonts.gstatic.com', crossOrigin: 'anonymous' },
      { rel: 'dns-prefetch', href: 'https://api.example.com' },
    ],
  },
};

// Component-level preloading
'use client';

import { useEffect } from 'react';

export function PrefetchLink({ href, children }: { href: string; children: React.ReactNode }) {
  useEffect(() => {
    const link = document.createElement('link');
    link.rel = 'prefetch';
    link.href = href;
    document.head.appendChild(link);
    
    return () => {
      document.head.removeChild(link);
    };
  }, [href]);
  
  return <a href={href}>{children}</a>;
}

// Priority hints for images
import Image from 'next/image';

export function HeroSection() {
  return (
    <Image
      src="/hero.jpg"
      alt="Hero"
      priority // Adds fetchpriority="high"
      loading="eager"
      sizes="100vw"
      className="w-full"
    />
  );
}

// Prefetching data with SWR/React Query
'use client';

import useSWR from 'swr';

function usePrefetch(url: string) {
  const { mutate } = useSWR(url);
  
  const prefetch = () => {
    mutate(); // Trigger prefetch
  };
  
  return { prefetch };
}

// Hover prefetching
function LinkWithPrefetch({ href, children }: { href: string; children: React.ReactNode }) {
  const router = useRouter();
  
  return (
    <a 
      href={href}
      onMouseEnter={() => router.prefetch(href)}
      onFocus={() => router.prefetch(href)}
    >
      {children}
    </a>
  );
}
```

### HTTP/2 Server Push (via Link headers)

Preemptive resource delivery:

```typescript
// middleware.ts
import { NextResponse } from 'next/server';

export function middleware(request: NextRequest) {
  const response = NextResponse.next();
  
  // Hint to server to push critical CSS
  response.headers.set(
    'Link',
    '</critical.css>; rel=preload; as=style, </font.woff2>; rel=preload; as=font'
  );
  
  return response;
}

// Critical CSS inlining
// app/layout.tsx
export default function RootLayout({ children }: { children: React.ReactNode }) {
  return (
    <html>
      <head>
        <style dangerouslySetInnerHTML={{ __html: criticalCss }} />
      </head>
      <body>{children}</body>
    </html>
  );
}

const criticalCss = `
  /* Critical above-fold styles */
  body { margin: 0; font-family: system-ui; }
  .header { height: 60px; background: #000; }
`;
```

## 52.6 CPU Profiling and Optimization

Identifying JavaScript execution bottlenecks.

### Performance Measurement

Using User Timing API:

```typescript
'use client';

import { useEffect, useCallback } from 'react';

// Custom hook for performance measurement
function usePerformanceMark(name: string) {
  useEffect(() => {
    performance.mark(`${name}-start`);
    
    return () => {
      performance.mark(`${name}-end`);
      performance.measure(name, `${name}-start`, `${name}-end`);
      
      const entries = performance.getEntriesByName(name);
      console.log(`${name} took ${entries[0].duration}ms`);
    };
  }, [name]);
}

// Measuring expensive operations
function ExpensiveComponent() {
  usePerformanceMark('ExpensiveComponent-render');
  
  const handleExpensiveOperation = useCallback(() => {
    const start = performance.now();
    
    // Expensive work
    const result = heavyComputation();
    
    const duration = performance.now() - start;
    
    // Send to analytics
    if (duration > 50) { // Long task threshold
      fetch('/api/performance-log', {
        method: 'POST',
        body: JSON.stringify({
          operation: 'heavyComputation',
          duration,
          timestamp: Date.now(),
        }),
      });
    }
    
    return result;
  }, []);
  
  return <button onClick={handleExpensiveOperation}>Compute</button>;
}

// Web Vitals monitoring
// app/_components/web-vitals.tsx
'use client';

import { useReportWebVitals } from 'next/web-vitals';

export function WebVitals() {
  useReportWebVitals((metric) => {
    // Send to analytics endpoint
    const body = JSON.stringify({
      name: metric.name,
      value: metric.value,
      id: metric.id,
    });
    
    // Use navigator.sendBeacon for reliability
    if (navigator.sendBeacon) {
      navigator.sendBeacon('/api/vitals', body);
    } else {
      fetch('/api/vitals', { body, method: 'POST', keepalive: true });
    }
  });
  
  return null;
}
```

### Long Task Optimization

Breaking up synchronous work:

```typescript
'use client';

// ❌ Bad: Blocking the main thread
function processLargeDataset(data: any[]) {
  const results = [];
  for (let i = 0; i < data.length; i++) {
    results.push(heavyComputation(data[i])); // Blocks for 500ms+
  }
  return results;
}

// ✅ Good: Yielding to main thread
async function processLargeDatasetChunked(data: any[]) {
  const results = [];
  const CHUNK_SIZE = 100;
  
  for (let i = 0; i < data.length; i += CHUNK_SIZE) {
    const chunk = data.slice(i, i + CHUNK_SIZE);
    
    // Process chunk
    results.push(...chunk.map(heavyComputation));
    
    // Yield control every chunk
    if (i + CHUNK_SIZE < data.length) {
      await new Promise(resolve => setTimeout(resolve, 0));
    }
  }
  
  return results;
}

// Using requestIdleCallback for non-critical work
function useIdleCallback(callback: () => void, deps: any[]) {
  useEffect(() => {
    if ('requestIdleCallback' in window) {
      const id = requestIdleCallback(callback, { timeout: 2000 });
      return () => cancelIdleCallback(id);
    } else {
      // Fallback
      const id = setTimeout(callback, 1);
      return () => clearTimeout(id);
    }
  }, deps);
}

// Web Workers for heavy computation
// hooks/use-worker.ts
export function useWorker<T, R>(workerScript: string) {
  const [result, setResult] = useState<R | null>(null);
  const [processing, setProcessing] = useState(false);
  const workerRef = useRef<Worker | null>(null);
  
  useEffect(() => {
    workerRef.current = new Worker(workerScript);
    workerRef.current.onmessage = (e) => {
      setResult(e.data);
      setProcessing(false);
    };
    
    return () => workerRef.current?.terminate();
  }, [workerScript]);
  
  const process = useCallback((data: T) => {
    setProcessing(true);
    workerRef.current?.postMessage(data);
  }, []);
  
  return { process, result, processing };
}

// workers/sorting.worker.ts
self.onmessage = (e) => {
  const { data } = e;
  // Heavy sorting on worker thread
  const sorted = data.sort((a: any, b: any) => complexCompare(a, b));
  self.postMessage(sorted);
};
```

## 52.7 Virtual Scrolling

Rendering massive lists without DOM overhead.

### Windowing Implementation

Rendering only visible items:

```typescript
'use client';

import { useRef, useState, useEffect, useCallback } from 'react';

interface VirtualListProps<T> {
  items: T[];
  itemHeight: number;
  height: number;
  renderItem: (item: T, index: number) => React.ReactNode;
  overscan?: number;
}

export function VirtualList<T>({ 
  items, 
  itemHeight, 
  height, 
  renderItem,
  overscan = 5 
}: VirtualListProps<T>) {
  const [scrollTop, setScrollTop] = useState(0);
  const containerRef = useRef<HTMLDivElement>(null);
  
  // Calculate visible range
  const totalHeight = items.length * itemHeight;
  const startIndex = Math.max(0, Math.floor(scrollTop / itemHeight) - overscan);
  const endIndex = Math.min(
    items.length,
    Math.ceil((scrollTop + height) / itemHeight) + overscan
  );
  
  const visibleItems = items.slice(startIndex, endIndex);
  const offsetY = startIndex * itemHeight;
  
  const handleScroll = useCallback((e: React.UIEvent<HTMLDivElement>) => {
    setScrollTop(e.currentTarget.scrollTop);
  }, []);
  
  return (
    <div
      ref={containerRef}
      onScroll={handleScroll}
      style={{ height, overflow: 'auto' }}
    >
      <div style={{ height: totalHeight, position: 'relative' }}>
        <div style={{ transform: `translateY(${offsetY}px)` }}>
          {visibleItems.map((item, index) => (
            <div key={startIndex + index} style={{ height: itemHeight }}>
              {renderItem(item, startIndex + index)}
            </div>
          ))}
        </div>
      </div>
    </div>
  );
}

// Usage with React Window (production library)
import { FixedSizeList as List } from 'react-window';

function LargeUserList({ users }: { users: User[] }) {
  const Row = useCallback(({ index, style }: { index: number; style: any }) => (
    <div style={style} className="flex items-center p-4 border-b">
      <img src={users[index].avatar} className="w-10 h-10 rounded-full mr-4" />
      <div>
        <div className="font-bold">{users[index].name}</div>
        <div className="text-gray-500">{users[index].email}</div>
      </div>
    </div>
  ), [users]);
  
  return (
    <List
      height={600}
      itemCount={users.length}
      itemSize={72}
      width="100%"
    >
      {Row}
    </List>
  );
}

// Dynamic height virtualization
import { VariableSizeList } from 'react-window';

function DynamicHeightList({ items }: { items: Array<{ id: string; content: string }> }) {
  const listRef = useRef<VariableSizeList>(null);
  
  // Measure item heights
  const getItemSize = useCallback((index: number) => {
    // Estimate based on content length
    return Math.max(100, items[index].content.length / 2);
  }, [items]);
  
  return (
    <VariableSizeList
      ref={listRef}
      height={600}
      itemCount={items.length}
      itemSize={getItemSize}
      width="100%"
    >
      {({ index, style }) => (
        <div style={style} className="p-4 border-b">
          <p>{items[index].content}</p>
        </div>
      )}
    </VariableSizeList>
  );
}
```

### Grid Virtualization

Two-dimensional windowing:

```typescript
'use client';

interface VirtualGridProps<T> {
  items: T[];
  columns: number;
  rowHeight: number;
  height: number;
  renderCell: (item: T, row: number, col: number) => React.ReactNode;
}

export function VirtualGrid<T>({ 
  items, 
  columns, 
  rowHeight, 
  height, 
  renderCell 
}: VirtualGridProps<T>) {
  const [scrollTop, setScrollTop] = useState(0);
  const totalRows = Math.ceil(items.length / columns);
  const totalHeight = totalRows * rowHeight;
  
  const startRow = Math.floor(scrollTop / rowHeight);
  const endRow = Math.min(
    totalRows,
    Math.ceil((scrollTop + height) / rowHeight)
  );
  
  const visibleItems = [];
  for (let row = startRow; row < endRow; row++) {
    for (let col = 0; col < columns; col++) {
      const index = row * columns + col;
      if (index < items.length) {
        visibleItems.push({ item: items[index], row, col, index });
      }
    }
  }
  
  return (
    <div 
      style={{ height, overflow: 'auto' }}
      onScroll={(e) => setScrollTop(e.currentTarget.scrollTop)}
    >
      <div style={{ height: totalHeight, position: 'relative' }}>
        {visibleItems.map(({ item, row, col, index }) => (
          <div
            key={index}
            style={{
              position: 'absolute',
              top: row * rowHeight,
              left: `${(col / columns) * 100}%`,
              width: `${100 / columns}%`,
              height: rowHeight,
            }}
          >
            {renderCell(item, row, col)}
          </div>
        ))}
      </div>
    </div>
  );
}
```

## Key Takeaways from Chapter 52

1. **React Compiler**: Enable the React Compiler to automatically memoize components without manual `useMemo` and `useCallback`. Write naturally without worrying about referential equality—the compiler adds memoization at build time. Use `'use no memo'` directives only for edge cases where manual control is necessary.

2. **Concurrent Rendering**: Use `useTransition` for non-urgent updates (filtering large lists, switching tabs) to keep the UI responsive during heavy renders. Implement `useDeferredValue` to show stale data during transitions, reducing perceived loading time and preventing jank in high-frequency update scenarios.

3. **Memory Leak Prevention**: Always return cleanup functions from `useEffect` for subscriptions, intervals, and event listeners. Use `AbortController` for fetch cancellation when components unmount. Leverage `WeakMap` for caches that shouldn't prevent garbage collection of the underlying objects.

4. **Bundle Optimization**: Configure `sideEffects: false` in `package.json` to enable aggressive tree-shaking. Use named exports instead of default exports containing multiple methods. Analyze bundles using `@next/bundle-analyzer` and split heavy dependencies using dynamic `import()` with preload triggers on hover/focus.

5. **Network Optimization**: Add `preconnect` hints to critical third-party origins (CDNs, APIs) in document `<head>`. Use `priority` prop on Next.js Images for LCP images to add `fetchpriority="high"`. Implement route prefetching on hover for instant navigation, and use `dns-prefetch` for resources likely to be needed on subsequent pages.

6. **CPU Profiling**: Instrument expensive operations using the User Timing API (`performance.mark/measure`) to identify long tasks (>50ms) blocking the main thread. Break up synchronous work into chunks using `setTimeout(0)` or `requestIdleCallback`. Offload heavy computation to Web Workers for true parallelism without blocking UI updates.

7. **Virtual Scrolling**: Implement windowing for lists exceeding 100 items using libraries like `react-window` or custom virtualization calculating visible ranges based on `scrollTop`. Render only items in viewport plus overscan buffer (5-10 items), using absolute positioning to maintain scroll position without thousands of DOM nodes.

## Coming Up Next

**Chapter 53: WebAssembly Integration**

Pushing performance boundaries further, Chapter 53 explores integrating WebAssembly (WASM) into Next.js applications. You'll learn to compile Rust or C++ to WASM for compute-intensive tasks, bridge JavaScript and WASM memory for efficient data transfer, implement image processing and cryptography in native code, and optimize bundle sizes for WASM modules. Understand when WASM outperforms JavaScript and how to gracefully degrade for unsupported environments.